In [1]:
# Importing relevant packages 
#!pip3 install python-Levenshtein --user
#!pip3 install fuzzywuzzy --user

import fuzzywuzzy
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [ ]:
#### Cleaning of the raw data 
#df1 = pd.read_csv('H:/data/2022_NETS_database/NETS2022_Headquarters/NETS2022_HQCompany.txt', delimiter='\t', encoding='latin1')
#df1 = df1.drop_duplicates(subset=['HQDuns', 'HQCompany', 'HQTradeName'])
#df1 = df1[['HQDuns', 'HQCompany', 'HQTradeName']]

In [2]:
# Uploading the data set 
#df1 = pd.read_csv('H:/data/2022_NETS_database/NETS2022_Headquarters/NETS2022_HQCompany.txt', delimiter='\t', encoding='latin1')
df1 = pd.read_stata('H:/InnovationProject/data/temp/NETS_prep.dta') 

In [3]:
df1['hqcompany'] = df1['hqcompany'].str.strip().str.lower().str.replace(r'[^\w\s]', '', regex=True)
df1['hqtradename'] = df1['hqtradename'].str.strip().str.lower().str.replace(r'[^\w\s]', '', regex=True)
#df_clean = df1.dropna(subset=['CZ_depagri_1990'])

In [4]:
df1.head()

,dunsnumber,CZ_depagri_1990,hqduns,hqcompany,hqtradename
0,1025899,20500.0,1025873,bicknell fuller paper box co,bicknell fuller corrugated
1,1025899,20500.0,1025873,bicknell fller ppr box cmpany,
2,1025899,20500.0,1025873,bicknell fuller ppr box co,bicknell fuller corrugated
3,1051044,20500.0,1005750,key polymer corporation,
4,1072800,20500.0,1068386,united die corp,knoxnorth shore die co


In [5]:
# Uploading the inventor data set 
df2 = pd.read_stata('H:/InnovationProject/data/temp/patentdata_clean_assignee.dta')
#df2 = df2[['assignee_id', 'county_fips_inventor', 'assignee_std']]
#df2 = df2.drop_duplicates(subset=['assignee_id', 'county_fips_inventor'])
#df2 = df2.rename(columns={'county_fips_inventor': 'county_fips'})

In [12]:
# Cleaning the data set 
length1 = 2
length2 = 3

# Fill var1 and var2 with leading zeros and combine them
df2['combined_var'] = df2['state_fips_inventor'].astype(str).str.zfill(length1) + df2['county_fips_inventor'].astype(str).str.zfill(length2)
df2 = df2.rename(columns={'combined_var': 'county_fips'})
df2 =df2[['assignee_id', 'assignee_std', 'gvkey', 'county_fips']]
df2.head()

,assignee_id,assignee_std,gvkey,county_fips
0,ab016f9a-90ed-48ef-9763-4e33a635c921,RAYTHEON COMPANY,010983,06037
1,e9480a97-61bb-49e9-b601-48becf420576,MILWAUKEE ELECTRIC TOOL CORPORATION,,55089
2,e9480a97-61bb-49e9-b601-48becf420576,MILWAUKEE ELECTRIC TOOL CORPORATION,,55089
3,e9480a97-61bb-49e9-b601-48becf420576,MILWAUKEE ELECTRIC TOOL CORPORATION,,55079
4,e9480a97-61bb-49e9-b601-48becf420576,MILWAUKEE ELECTRIC TOOL CORPORATION,,55079


In [13]:
# Merging in the information on the commuting zones 
df4 = pd.read_stata('H:/InnovationProject/data/raw/var_CommutingZones/CZ_combined.dta')
df4 = df4[['county_fips', 'CZ_depagri_1990']]

length1 = 5
df4['county_fips'] = df4['county_fips'].astype(str).str.zfill(length1)
df4.head()

,county_fips,CZ_depagri_1990
0,01001,11101.0
1,01003,11001.0
2,01005,10301.0
3,01007,10801.0
4,01009,10700.0


In [14]:
# Perform a many-to-one merge on 'county_fips_inventor'
df2 = pd.merge(df4, df2, on=['county_fips'])
df2.head()

,county_fips,CZ_depagri_1990,assignee_id,assignee_std,gvkey
0,01001,11101.0,f1257244-bbba-439f-ad6f-315307608af4,RHEEM MANUFACTURING COMPANY,
1,01001,11101.0,2b5ed62c-a3ac-4597-87f3-4b0b2d602577,BASF SE,017436
2,01001,11101.0,7ba73a07-6ed4-4f95-a2db-2f9a8334d7e8,THE GOVERNMENT OF THE UNITED STATES OF AMERICA...,
3,01001,11101.0,f1257244-bbba-439f-ad6f-315307608af4,RHEEM MANUFACTURING COMPANY,
4,01001,11101.0,f1257244-bbba-439f-ad6f-315307608af4,RHEEM MANUFACTURING COMPANY,


In [15]:
# Cleaning the data set and only keeping necessary information
df2 = df2[['assignee_id', 'assignee_std', 'CZ_depagri_1990']]
df2 = df2.drop_duplicates(subset=['assignee_id', 'CZ_depagri_1990'])

In [16]:
# Cleaning the data sets in a similar fashion
df2['assignee_std'] = df2['assignee_std'].str.strip().str.lower().str.replace(r'[^\w\s]', '', regex=True)
df2.head()

,assignee_id,assignee_std,CZ_depagri_1990
0,f1257244-bbba-439f-ad6f-315307608af4,rheem manufacturing company,11101.0
1,2b5ed62c-a3ac-4597-87f3-4b0b2d602577,basf se,11101.0
2,7ba73a07-6ed4-4f95-a2db-2f9a8334d7e8,the government of the united states of america...,11101.0
5,10c5947f-58fb-41f9-935c-62b103b93112,neptune technology group inc,11101.0
9,09988f26-97bb-46fe-b46a-7fef4f3c0e1d,intelligence designs llc,11101.0


In [18]:
# Counting occurrence of commuting zone value 19600

count_value = 19600

count = (df2['CZ_depagri_1990'] == count_value).sum()
print(f"The value {count_value} appears {count} times in the 'CZ_depagri_1990' column.")

The value 19600 appears 15106 times in the 'CZ_depagri_1990' column.


In [20]:
# Merging both data sets together based on direct matching 
#for value in df2['CZ_depagri_1990'].unique():
value = 196000
    df2_subset = df2[df2['CZ_depagri_1990'] == value]
    df1_subset = df1[df1['CZ_depagri_1990'] == value ]

In [25]:
# Step 1: Direct Matching 
df2_subset = df2_subset.rename(columns={'assignee_std': 'hqcompany'})
matched1 = pd.merge(df1_subset, df2_subset, on=['hqcompany', 'CZ_depagri_1990'], how='inner')
matched1 

,dunsnumber,CZ_depagri_1990,hqduns,hqcompany,hqtradename,assignee_id
0,7737839,19600.0,107212169,verizon communications inc,verizon,675350cb-b7ed-4d8d-a292-126d40513486
1,7737839,19600.0,107212169,verizon communications inc,verizon communications,675350cb-b7ed-4d8d-a292-126d40513486
2,1864016,19600.0,107212169,verizon communications inc,verizon,675350cb-b7ed-4d8d-a292-126d40513486
3,1864016,19600.0,107212169,verizon communications inc,verizon communications,675350cb-b7ed-4d8d-a292-126d40513486
4,9435078,19600.0,107212169,verizon communications inc,verizon,675350cb-b7ed-4d8d-a292-126d40513486
...,...,...,...,...,...,...
4106,8931514,19600.0,151064821,staples inc,staplesthe office superstore,ff9499bc-63bf-463d-8c4d-28c527e88ea8
4107,4003912,19600.0,47021092,mallinckrodt llc,mallinckrodt pharmaceuticals,a9c85934-e762-4ad8-99e0-be34cf6d5d3e
4108,2163178,19600.0,2163178,transistor devices inc,astrodyne tdi,2780e0bb-abb3-42ec-8089-f89381331ca1
4109,2163178,19600.0,2163178,transistor devices inc,t d i power,2780e0bb-abb3-42ec-8089-f89381331ca1


In [26]:
# Step 2: Fuzzy Matching

# Identify direct matches:Merge df1 with matched and keep rows that are not in the matched DataFrame

df1_filtered = df1_subset.merge(matched1, on=['hqcompany'], how='left', indicator=True)
df1_filtered = df1_filtered[df1_filtered['_merge'] == 'left_only'].drop('_merge', axis=1)

# Merge df2 with matched and keep rows that are not in the matched DataFrame
df2_filtered = df2_subset.merge(matched1, on=['hqcompany'], how='left', indicator=True)
df2_filtered = df2_filtered[df2_filtered['_merge'] == 'left_only'].drop('_merge', axis=1)


In [34]:
df2_filtered.head()

,assignee_id,hqcompany,CZ_depagri_1990
0,486f09aa-157e-40f9-bc86-0c3508110919,alex toys llc,19600.0
1,2b5ed62c-a3ac-4597-87f3-4b0b2d602577,basf se,19600.0
2,777cea06-b934-490c-9cba-7baddb3c7bec,opko health inc,19600.0
3,1bead90f-1cc7-4b00-8898-f51aeb28e0f5,the trustees of columbia university in the cit...,19600.0
4,131272b5-c296-4ef9-b2ae-2e34e9055490,international business machines corporation,19600.0


In [35]:
# Renaming the matched variables 
df2_filtered = df2_filtered[['assignee_id_x', 'hqcompany', 'CZ_depagri_1990_x']]
df2_filtered = df2_filtered.rename(columns={'assignee_id_x': 'assignee_id'})
df2_filtered = df2_filtered.rename(columns={'CZ_depagri_1990_x': 'CZ_depagri_1990'})
df1_filtered = df1_filtered[['dunsnumber_x', 'hqduns_x', 'hqcompany', 'hqtradename_x', 'CZ_depagri_1990_x']]
df1_filtered = df1_filtered.rename(columns={'dunsnumber_x': 'dunsnumber'})
df1_filtered = df1_filtered.rename(columns={'hqtradename_x': 'hqtradename'})
df1_filtered = df1_filtered.rename(columns={'hqduns_x': 'hqduns'})
df1_filtered = df1_filtered.rename(columns={'CZ_depagri_1990_x': 'CZ_depagri_1990'})

KeyError: "['assignee_id_x', 'CZ_depagri_1990_x'] not in index"

In [33]:
# Fuzzy Matching
def match_names(row, df2_filtered, threshold=90):
    name = row['hqcompany']
    matches = process.extractOne(name, df2_filtered['hqcompany'], scorer=fuzz.token_sort_ratio)
    if matches[1] >= threshold:
        return matches[0]
    return None

df1_filtered['matched_name'] = df1_filtered.apply(match_names, axis=1, df2_filtered=df2_filtered)
df1_filtered.head()

,dunsnumber,hqduns,hqcompany,hqtradename,CZ_depagri_1990,matched_name
24,19935535,34387894,tnt usa inc,tnt express worldwide,19600.0,None
25,19935535,34387894,tnt usa inc,tnt express,19600.0,None
27,6986517,6986517,kroll boris fabrics inc,,19600.0,None
28,1489319,1212844,novo corporation,,19600.0,noxso corporation
29,3373719,3373719,dsv solutions llc,,19600.0,None


In [80]:
# Renaming the matched companies 
df1_filtered = df1_filtered.rename(columns={'hqcompany': 'original_name'})
df1_filtered = df1_filtered.rename(columns={'matched_name': 'hqcompany'}) 

In [66]:
# Merging with dataframe 2 
df1_filtered = df1_filtered.dropna(subset=['matched_name'])
matched2 = pd.merge(df2_filtered, df1_filtered, on=['hqcompany', 'CZ_depagri_1990'], how='inner')
matched2

In [81]:
# Combining both dataframes 
df_combined = pd.concat([matched1, matched2], ignore_index=True)
# Missing Original Name indicates that there was a direct match 
df_combined.to_csv('H:/InnovationProject/data/temp/matching_NETS/match_19600.csv') 

# Export the data for which this did not yield a match 
df1_cleaned = df1_filtered.applymap(lambda x: x.encode('latin-1', 'replace').decode('latin-1') if isinstance(x, str) else x)
df2_cleaned = df2_filtered.applymap(lambda x: x.encode('latin-1', 'replace').decode('latin-1') if isinstance(x, str) else x)

df1_cleaned.to_stata('H:/InnovationProject/data/temp/matching_NETS/nonmatch_NETS_19600.dta') 
df2_cleaned.to_stata('H:/InnovationProject/data/temp/matching_NETS/nonmatch_patents_19600.dta') 